In [1]:
import cdsapi
from netCDF4 import Dataset
import numpy as np
from shapely.geometry import Point
from shapely.prepared import prep
import geopandas
from datetime import datetime, timedelta
import os

In [ ]:
#get the data 
dataset = "reanalysis-era5-single-levels"
request = {"product_type": ["reanalysis"], "variable": ["2m_dewpoint_temperature", "2m_temperature"], "year": ["2024", "2025"], "month": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"], "day": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31"], "time": ["00:00", "01:00", "02:00", "03:00", "04:00", "05:00", "06:00", "07:00", "08:00", "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00", "20:00", "21:00", "22:00", "23:00"], "data_format": "netcdf", "download_format": "zip", "area": [51, -5, 41.5, 9.75]}
client = cdsapi.Client()
client.retrieve(dataset, request, "2024-2025.zip")

In [ ]:
#for data of shape (n, x, y) with n the hour, x the latitude and y the longitude 
#we can find daily datas in the form (0:23, x, y), (24:47, x, y), (48:71, x, y)....
#then make the calcalutions on the daily data, find the data points of each dep, find the day and finaly save the produces datas

In [2]:
geo = geopandas.read_file("data/geojsonfrance_corse_20.json") #read france departement geometries
geo["code"] = geo["code"].astype(int)
geo = geo.sort_values(by="code").reset_index(drop=True)

In [3]:
points = []
data_index = 0
points = dict()
#for every lat and lon, we make a dict of index POINT(lon, lat) and value the index of the data associated with this point
for lat in Dataset("data/ERA5/1940-1945.nc").variables["latitude"][:]:
        for lon in Dataset("data/ERA5/1940-1945.nc").variables["longitude"][:]:
            points[Point(lon, lat)] = data_index
            data_index +=1

: 

In [12]:
temp = Dataset("data/ERA5/1940-1945.nc").variables["t2m"][:]
dewP = Dataset("data/ERA5/1940-1945.nc").variables["d2m"][:]

date = datetime(1940, 1, 1)

# Calculate how many complete days we have
total_hours = temp.shape[0]
hours_per_day = 24
num_complete_days = total_hours // hours_per_day

result = []
# Loop through the array in 24-hour chunks
for day in range(num_complete_days):
    # Calculate start and end indices for this day
    start_idx = day * hours_per_day       # e.g., 0, 24, 48, ...
    end_idx = start_idx + hours_per_day   # e.g., 24, 48, 72, ...
    
    # Get the data for this day
    dailyTemp = temp[start_idx:end_idx]
    dailyDewP = dewP[start_idx:end_idx]

    rh = 100*(np.exp((17.625*dailyDewP)/(243.04+dailyDewP))/np.exp((17.625*dailyTemp)/(243.04+dailyTemp))) #August-Roche-Magnus equation to calculate relative humidity

    vpd = 6.1078 * (1 - rh / 100) * np.exp(17.08085 * dailyTemp / (234.175 + dailyTemp)) * 0.1 #vpd in kPa #Alduchov, O. A., and R. E. Eskridge, 1996 formula
    vpd_max = np.max(vpd, axis=0)
    vpd_max_flattened = vpd_max.flatten()
    vpd_min = np.min(vpd, axis=0)
    vpd_min_flattened = vpd_min.flatten()
    vpd_mean = np.mean(vpd, axis=0)
    vpd_mean_flattened = vpd_mean.flatten()

    for _, dep in geo.itterows():
        prepared = prep(dep["geometry"]) #use prep for batch operations
        valid_points = []
        valid_points.extend(filter(prepared.contains, points)) #find POINTS in dep
        valid_indices = [points[point] for point in valid_points if point in points] #make a list of valid points that are in the dep

        vpd_max_dep = vpd_max_flattened[valid_indices] #find values with indices
        vpd_mean_dep = vpd_mean_flattened[valid_indices] #find values with indices
        vpd_min_dep = vpd_min_flattened[valid_indices] #find values with indices

        result.append[{"date": date, "dep": dep["departement"], "vpd_max": vpd_max_dep, "vpd_mean": vpd_mean_dep, "vpd_min": vpd_min_dep}]
    
    date += timedelta(days=1)
    break

NameError: name 'geo' is not defined

In [4]:
temp = Dataset("data/2025test.nc").variables["t2m"][:] - 273.15
temp.shape

(24, 41, 61)

In [ ]:
temp = Dataset("data/2025test.nc").variables["t2m"][:] - 273.15 #convert to from Kelvin to Celsius
dewP = Dataset("data/2025test.nc").variables["d2m"][:] - 273.15 #convert to from Kelvin to Celsius
rh = 100*(np.exp((17.625*dewP)/(243.04+dewP))/np.exp((17.625*temp)/(243.04+temp))) #August-Roche-Magnus equation to calculate relative humidity

vpd = 6.1078 * (1 - rh / 100) * np.exp(17.08085 * temp / (234.175 + temp)) * 0.1 #vpd in kPa #Alduchov, O. A., and R. E. Eskridge, 1996 formula

#calculations and flattend datas
vpd_max = np.max(vpd, axis=0)
vpd_max_flattened = vpd_max.flatten()
vpd_min = np.min(vpd, axis=0)
vpd_min_flattened = vpd_min.flatten()
vpd_mean = np.mean(vpd, axis=0)
vpd_mean_flattened = vpd_mean.flatten()

In [ ]:
prepared = prep(geo["geometry"].iloc[0]) #use prep for batch operations
valid_points = []
valid_points.extend(filter(prepared.contains, points)) #find POINTS in dep
valid_indices = [points[point] for point in valid_points if point in points] #make a list of valid points that are in the dep

In [ ]:
vpd_max_flattened[valid_indices] #find values with indices

masked_array(data=[0.3330575227737427, 0.3533657491207123,
                   0.3058649003505707, 0.20237870514392853,
                   0.1655619740486145, 0.12627676129341125,
                   0.28827622532844543, 0.28825050592422485,
                   0.23285146057605743, 0.212148055434227,
                   0.1754762977361679, 0.2255707085132599,
                   0.15644685924053192],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False, False],
       fill_value=1e+20,
            dtype=float32)

In [138]:
vpd_min_flattened[valid_indices]

masked_array(data=[0.15823116898536682, 0.17255668342113495,
                   0.16261878609657288, 0.08930866420269012,
                   0.034297965466976166, 0.031788866966962814,
                   0.13649725914001465, 0.14891740679740906,
                   0.11739559471607208, 0.05278622731566429,
                   0.01027910690754652, 0.055440690368413925,
                   0.041455790400505066],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False, False],
       fill_value=1e+20,
            dtype=float32)

In [139]:
vpd_mean_flattened[valid_indices]

masked_array(data=[0.20956015586853027, 0.2174536188443502,
                   0.21111108859380087, 0.1379851500193278,
                   0.10079453388849895, 0.0886037250359853,
                   0.20597984393437704, 0.21295567353566489,
                   0.1918415625890096, 0.14592679341634116,
                   0.11190817753473918, 0.16238168875376383,
                   0.11925611893335979],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False, False],
       fill_value=1e+20)